In [1]:
import sys; sys.path.append('..')

In [2]:
from os import path
import random

In [3]:
import pandas as pd

In [4]:
from graph import Graph
from graph.reader import read_problem

In [5]:
problema = path.join('..', 'datasets', 'ORLibrary', 'steinc1.txt')
# problema = path.join('..', 'datasets', 'ORLibrary', 'steinb16.txt')

STPG = read_problem(problema)

In [6]:
assert len(STPG.terminals) == STPG.nro_terminals

STPG.nro_terminals

5

In [7]:
STPG.nro_nodes

500

In [8]:
STPG.nro_edges

625

In [9]:
print(STPG.terminals)

{128, 164, 13, 243, 499}


---

## Algoritmo Genético

In [10]:
from base.binary.combiner import crossover_1point, crossover_2points, crossover_uniform
from base.chromosome import random_binary
from base.condition import BestKnownReached, BestSolutionKnowReached, Stagnation
from base.customevol import SteinerEvolution as Evolution
from base.customevol import SteinerPopulation as Population
from base.mutate import flip_onebit
from base.normalization import normalize
from base.pickers import random_picker
from base.selector import roullete
from base.tracker import DataTracker
from base.util import display, update_best, update_generation

from treetools import Eval, Converter

In [11]:
tamanho_populacao = 50
comprimento_cromosso = STPG.nro_nodes - STPG.nro_terminals

tx_mutacao = 0.2
prob_troca_gene = 0.6

In [12]:
funcao_avaliacao = Eval(STPG)

In [13]:
populacao = (Population(chromosomes=[random_binary(comprimento_cromosso) for _ in range(tamanho_populacao) ],
                        eval_function=funcao_avaliacao,
                        maximize=True)
                        .evaluate()
                        .callback(normalize)
                        .callback(update_best))

In [14]:
len(populacao)

50

In [15]:
evolution = (Evolution()
                .evaluate()
                .callback(normalize)
                .callback(update_best)
                .select(selection_func=roullete)
                .crossover(combiner=crossover_uniform, 
                           parent_picker=random_picker, 
                           pbcrossover=prob_troca_gene)
                .mutate(mutate_function=flip_onebit, 
                        probability=tx_mutacao)
                .callback(update_generation)
                .callback(display, every=100))

In [16]:
%%time

result = populacao.evolve(evolution, n=500)

Population 3774a7 | size 50 | generation 100
Population 3774a7 | size 50 | generation 200
Population 3774a7 | size 50 | generation 300
Population 3774a7 | size 50 | generation 400
Population 3774a7 | size 50 | generation 500
Wall time: 2min 47s


In [17]:
result.runtime

167.69630790000002

In [18]:
result.documented_best.cost

1099

In [19]:
result.documented_best.last_improvement

496

In [20]:
result.documented_best.chromosome

'111110000010001111110000100111110010010000000010010011100000000000001100110011001001001011100100100010010000011010100110011111001111010101101110000001001010001100001011111001010000001000100111000110001110110001001101110100000001000101100101111011011000011011000010110100001110010000000101001011001001100001001010010001010001110001111100001001100010010011001100100011011100010100000101110011111101100001000011010101000110100010100001110100110000010100010001010011101101011010001101101011110001001'

In [21]:
assert len(result.documented_best.chromosome) == comprimento_cromosso

len(result.documented_best.chromosome)

495

---

In [22]:
# individuos = sorted(result, key=lambda x: x.cost )

individuos = list(result)

In [23]:
custos = [ individuo.cost for individuo in individuos ]

print(custos)

[1122, 1134, 1143, 1228, 1133, 1115, 1205, 1138, 1128, 1214, 1131, 1223, 1116, 1426, 1114, 1132, 1247, 1138, 1120, 1118, 1222, 1128, 1129, 1139, 1120, 1137, 1129, 1146, 1133, 1125, 1138, 1130, 1126, 1124, 1128, 1123, 1140, 1235, 1135, 1136, 1118, 1135, 1124, 1137, 1128, 1127, 1121, 1233, 1129, 1139]


In [24]:
type(individuos)

list

In [25]:
converter = Converter(STPG)

In [26]:
individuos = [converter.binary2treegraph(individuo) for individuo in individuos ]

In [27]:
for individuo in individuos:
    individuo.evaluate(eval_function=funcao_avaliacao)

In [28]:
parent = individuos[0]

parent.chromosome.edges

defaultdict(dict,
            {2: {365: 3, 489: 9},
             3: {221: 1, 223: 4, 380: 8},
             4: {48: 1, 112: 9},
             5: {217: 5},
             11: {336: 2, 179: 8, 440: 9},
             13: {374: 2},
             16: {418: 7},
             17: {437: 6, 177: 8},
             18: {193: 9},
             19: {78: 1, 164: 8, 231: 10},
             20: {90: 1, 401: 2, 487: 5},
             21: {173: 2, 278: 7},
             26: {372: 4, 378: 7},
             29: {120: 3},
             30: {397: 1, 398: 4, 346: 5, 133: 6, 259: 6},
             31: {313: 5, 126: 9, 388: 10},
             32: {209: 3, 429: 5, 452: 5, 179: 6, 491: 6, 295: 7, 330: 9},
             33: {216: 2, 171: 4, 315: 4, 334: 5, 195: 9},
             36: {220: 4},
             39: {411: 2, 401: 4},
             48: {4: 1},
             51: {373: 2, 209: 4, 102: 8, 126: 9},
             54: {423: 1},
             55: {482: 2},
             56: {318: 8, 204: 9},
             70: {172: 3, 436: 3, 112: 8},

In [29]:
parent.is_connected

True

In [30]:
parent.qtd_partitions

1

---

In [31]:
import networkx as nx
from matplotlib import pyplot as plt
import math

from draw import hierarchy_pos

In [32]:
def convert_graph(graph, color='black'):
    G = nx.Graph()
    for v, value in graph.items():
        G.add_node(v)
        for w, weight in value.items():
            if w in G[v]:
                continue
            G.add_edge(v, w, weight=weight, color=color)
    return G    

In [33]:
def plotar_(individuo, raiz):
    
    grafo = convert_graph(individuo.chromosome.edges)
    
    
    pos = hierarchy_pos(grafo, root=raiz, width = 2*math.pi, xcenter=0)
    
    pos_t = {u: (r * math.cos(theta), r * math.sin(theta)) for u, (theta, r) in pos.items()}
    
    nx.draw(grafo,
        pos_t,
        node_color='#A0CBE2',
        edge_color='black',
        width=2, 
        node_size=100,
        with_labels=True)

    nx.draw_networkx_nodes(grafo,
                           pos=pos_t,
                           nodelist = STPG.terminals,
                           node_color = 'red', 
                           node_size = 150)

    plt.show()

In [35]:
# plotar_(individuos[5], 87)

In [36]:
# plotar_(individuos[6], 87)

---

In [37]:
def nro_vertices(individuo):
    return len(individuo.chromosome.vertices)

def nro_arestas(individuo):
    return len(list(individuo.chromosome.gen_undirect_edges()))

In [38]:
def vertices_semelhantes(A, B):
    contador = 0
    grafo_A = A.chromosome
    grafo_B = B.chromosome
    for vertice_a in grafo_A.vertices:
        if vertice_a in grafo_B:
            contador += 1
        
    return contador

In [39]:
def arestas_semelhantes(A, B):
    contador = 0
    grafo_a = A.chromosome
    grafo_b = B.chromosome
    
    for edge in grafo_a.gen_undirect_edges():
        if grafo_b.has_edge(*edge):
            contador += 1
    return contador

In [40]:
def custo_(individuo):
    return individuo.cost

In [41]:
random.shuffle(individuos)

for A, B in zip(individuos[0::2], individuos[1::2]):
    print(A.cost, B.cost)

1138 1122
1121 1128
1114 1125
1131 1134
1118 1135
1128 1143
1132 1122
1124 1137
1118 1128
1133 1146
1128 1124
1133 1129
1120 1138
1115 1105
1136 1126
1138 1120
1147 1139
1128 1135
1114 1133
1123 1139
1129 1116
1127 1130
1123 1140
1137 1226
1135 1129


In [42]:
dados = list()

for A, B in zip(individuos[0::2], individuos[1::2]):
    registro = [
        custo_(A),
        custo_(B),
        nro_vertices(A),
        nro_vertices(B),
        vertices_semelhantes(A, B),
        nro_arestas(A),
        nro_arestas(B),
        arestas_semelhantes(A,B)
    ]
    
    dados.append(registro)

In [43]:
colunas = [
    'c_a',
    'c_b',
    'V_a',
    'V_b',
    'V_sem',
    'E_a',
    'E_b',
    'E_sem'
]

In [44]:
frame = pd.DataFrame(data=dados,columns=colunas)

In [45]:
frame

,c_a,c_b,V_a,V_b,V_sem,E_a,E_b,E_sem
0,1138,1122,224,221,218,223,219,216
1,1121,1128,222,223,218,221,221,215
2,1114,1125,219,223,219,218,222,216
3,1131,1134,222,226,221,221,225,217
4,1118,1135,221,223,220,220,222,217
5,1128,1143,226,225,224,225,224,220
6,1132,1122,222,222,219,221,221,218
7,1124,1137,223,224,221,222,223,219
8,1118,1128,221,223,219,220,222,218
9,1133,1146,225,224,221,224,223,217


---

## Verificar partições

In [46]:
from graph.util import has_cycle
from itertools import product

In [47]:
def gg_union(individuo_a, individuo_b):
    '''Return the union graph'''
    
    A = individuo_a.chromosome
    B = individuo_b.chromosome

    C = Graph()

    for v, u in A.gen_undirect_edges():
        w = A[v][u]
        C.add_edge(v,u,weight=w)

    for v, u in B.gen_undirect_edges():
        if not C.has_edge(v,u):
            w = B[v][u]
            C.add_edge(v,u,weight=w)

    return C

In [48]:
def partitioning(individuo_a, individuo_b):
    
    subtree_a = individuo_a.chromosome
    subtree_b = individuo_b.chromosome

    partition = Graph()
    offspring = Graph()
    
    for v, w in subtree_a.gen_undirect_edges():
        weight = subtree_a.weight(v,w)

        if subtree_b.has_edge(v, w):
            offspring.add_edge(v, w, weight=weight)
        else:
            partition.add_edge(v, w, weight=weight)
            
    for v, w in subtree_b.gen_undirect_edges():
        weight = subtree_b.weight(v,w)

        if not subtree_a.has_edge(v, w):
            partition.add_edge(v, w, weight=weight)
            
    return partition, offspring   

In [53]:
prod = product(individuos, repeat=2)

counter = 0
counter_cycle = 0
not_cycle = 0

for a, b in prod:
    p, _ = partitioning(a, b)
    counter += 1
    if has_cycle(p):
        counter_cycle += 1
    else:
        not_cycle += 1

In [54]:
counter

2500

In [55]:
counter_cycle

0

In [56]:
counter_cycle / counter

0.0

In [57]:
not_cycle

2500